In [2]:
# Imports
import os
import numpy as np
import pandas as pd

In [4]:
main_directory="./"

In [229]:
import os
import numpy as np
import pandas as pd

# Main directory containing the CSV files
main_directory = "./"

# Define a function to calculate the tag
def get_tag(address):
    tag_hex = hex(int(address,16) >> 17)
    return tag_hex

def get_offset(address):
    offset = hex(int(address,16)&0x3f)
    return offset

def get_3tag_1(tag):
    three_tag = hex(int(tag,16) & 0x7) 
    return three_tag

def get_3tag_2(tag):
    three_tag = hex((int(tag,16) & 0x38) >> 3) 
    return three_tag

def get_3tag_3(tag):
    three_tag = hex((int(tag,16) & 0x1c0)>> 6)
    return three_tag

# Process each CSV file
for file_name in os.listdir(main_directory):
    if file_name.endswith('313B_train.csv'):
        file_path = os.path.join(main_directory, file_name)
        output_filename = os.path.join(main_directory, os.path.splitext(file_name)[0] + '_features.csv')
        print(f"Processing file: {file_path}")

        # Read CSV file and slice rows
        df = pd.read_csv(file_path, header=None)
        
        df = df[1000000:1010000].reset_index(drop=True)  # Slice and reset index
        df.columns = ['pc', 'address', 'set', 'way', 'instr_type', 'recency', 'hit/miss'] #, 'hits_count', 'preuse_dist']
        print(df.head())

        # Get tag column
        df['tag'] = df['address'].apply(get_tag)
        address_index = df.columns.get_loc('address')  # Get the index of the address column
        df.insert(address_index + 1, 'tag', df.pop('tag'))  # Insert the tag column after the address column

        # Get offset column
        df['offset'] = df['address'].apply(get_offset)
        address_index = df.columns.get_loc('address')  # Get the index of the address column
        df.insert(address_index + 2, 'offset', df.pop('offset'))  # Insert the tag column after the address column

        df['3_tag_1'] = df['tag'].apply(get_3tag_1)
        df['3_tag_2'] = df['tag'].apply(get_3tag_2)
        df['3_tag_3'] = df['tag'].apply(get_3tag_3)

        # print (df)

        # Initialize results lists
        hits_count = []
        salm = []

        # Iterate through the DataFrame
        for i in range(df.shape[0]):
            current_tag = df.loc[i, 'tag']
            current_set = df.loc[i, 'set']

            # Initialize counters
            hit_count = 0
            salm_count = 0

            for j in range(i - 1, -1, -1):
                # Calculate hits_count
                if df.loc[j, 'tag'] == current_tag:
                    if df.loc[j, 'hit/miss'] == 1:
                        hit_count += 1
                    else:
                        break
            #Append
            hits_count.append(hit_count)

            for j in range(i - 1, -1, -1):    
                # Calculate salm
                if df.loc[j, 'hit/miss'] == 0:  
                        break
                else:
                    if df.loc[j, 'set'] == current_set:  
                        salm_count += 1 
            
            #Append
            salm.append(salm_count)

        # Add the results as new columns
        df['hits_count'] = hits_count
        df['salm'] = salm

        # Save the DataFrame to a new CSV file
        df.to_csv(output_filename, index=False)
        print(f"Processed file saved to: {output_filename}")
        #print(df)

Processing file: ./astar_313B_train.csv
         pc         address   set  way  instr_type  recency  hit/miss
0  0x4090c3  0xe6f224c31360  1101   11           0       15         0
1  0x409270  0xfc47cbc46380   398   15           0       15         0
2  0x4090c3  0x42a062b10fb0  1086   13           0       15         0
3  0x409270  0x44e52c205300   332    7           0       15         0
4  0x409270   0xc1b5f16b3b0   718   12           0       15         0
            pc         address         tag offset   set  way  instr_type  \
0     0x4090c3  0xe6f224c31360  0x73791261   0x20  1101   11           0   
1     0x409270  0xfc47cbc46380  0x7e23e5e2    0x0   398   15           0   
2     0x4090c3  0x42a062b10fb0  0x21503158   0x30  1086   13           0   
3     0x409270  0x44e52c205300  0x22729610    0x0   332    7           0   
4     0x409270   0xc1b5f16b3b0   0x60daf8b   0x30   718   12           0   
...        ...             ...         ...    ...   ...  ...         ...   
9995  0x

In [145]:
########## EXAMPLE #############

# Example DataFrame
data = {'address': ['0x12345678', '0xABCDEF01', '0x98765432']}
df = pd.DataFrame(data)

# Define a function to calculate the tag
def extract_tag(address):
    # Convert the address to an integer
    address_int = int(address, 16)
    # Right-shift the address by 11 (log2(2048)) + 6 (log2(64)) bits to ignore the last 11 bits
    tag_int = address_int >> 17
    # Convert the tag back to hexadecimal format
    tag_hex = hex(tag_int)
    return tag_hex

# Apply the function to the address column
df['tag'] = df['address'].apply(extract_tag)

# Insert the tag column after the address column
address_index = df.columns.get_loc('address')  # Get the index of the address column
df.insert(address_index + 1, 'tag', df.pop('tag'))  # Insert the tag column after the address column

# Display the resulting DataFrame
print(df)


      address     tag
0  0x12345678   0x91a
1  0xABCDEF01  0x55e6
2  0x98765432  0x4c3b


[time_stamp,pc,memory_address,cache_set,cache_way,instruction_type,recency,hit/miss]

In [158]:
# CODE FOR LINE HITS SINCE LAST INSERTION

############### EXAMPLE ##########

# Sample DataFrame
data = {
    'address':  [1, 2, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1],
    'hit/miss': [0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1]
}
df_trial = pd.DataFrame(data)

# Initialize a results list
results = []

# Iterate through the DataFrame
for i in range(df_trial.shape[0]):
    current_address = df_trial.loc[i, 'address']
    count = 0  # Initialize hit count for the current row
    
    # Iterate backwards to check previous rows for hits until a miss is found
    for j in range(i - 1, -1, -1):  # Loop from the previous row to the beginning
        if df_trial.loc[j, 'address'] == current_address:
            if df_trial.loc[j, 'hit/miss'] == 1:
                count += 1
            else:
                break  # Stop counting if a miss is encountered
        else:
            continue  # Skip rows with a different address

    # Append the count to the results
    results.append(count)

# Add the results as a new column in the DataFrame
df_trial['hits'] = results

print(df_trial)


    address  hit/miss  hits
0         1         0     0
1         2         1     0
2         3         0     0
3         1         1     0
4         2         0     1
5         1         1     1
6         2         1     0
7         1         1     2
8         2         0     1
9         1         0     3
10        2         1     0
11        1         1     0


In [228]:
# CODE FOR LINE HITS SINCE LAST INSERTION

# Initialize a results list
hits_count = []

# Iterate through the DataFrame
for i in range(df.shape[0]):
    current_address = df.loc[i, 'tag']
    count = 0  # Initialize hit count for the current row
    
    # Iterate backwards to check previous rows for hits until a miss is found
    for j in range(i - 1, -1, -1):  # Loop from the previous row to the beginning
        if df.loc[j, 'tag'] == current_address:
            if df.loc[j, 'hit/miss'] == 1:
                count += 1
            else:
                break  # Stop counting if a miss is encountered
        else:
            continue  # Skip rows with a different address

    # Append the count to the results
    hits_count.append(count)

# Add the results as a new column in the DataFrame
df['hits_count'] = hits_count

print(df)

            pc         address         tag offset   set  way  instr_type  \
0     0x4090c3  0xe6f224c31360  0x73791261   0x20  1101   11           0   
1     0x409270  0xfc47cbc46380  0x7e23e5e2    0x0   398   15           0   
2     0x4090c3  0x42a062b10fb0  0x21503158   0x30  1086   13           0   
3     0x409270  0x44e52c205300  0x22729610    0x0   332    7           0   
4     0x409270   0xc1b5f16b3b0   0x60daf8b   0x30   718   12           0   
...        ...             ...         ...    ...   ...  ...         ...   
9995  0x409270  0x2c2cbb4f6410  0x16165da7   0x10  1424    1           0   
9996  0x409228   0xab547476740   0x55aa3a3    0x0  1437   15           0   
9997  0x4090c3  0x6af9d6cd9f80  0x357ceb66    0x0  1662   11           0   
9998  0x4090c3  0x1b732561ebf0   0xdb992b0   0x30  1967   11           0   
9999  0x409270  0x71d5f3d96740  0x38eaf9ec    0x0  1437   10           0   

      recency  hit/miss 3_tag_1 3_tag_2 3_tag_3  hits_count  salm  
0          15      

In [225]:
# CODE FOR SET ACCESSES SINCE LAST MISS

# Initialize a results list
salm = []
    
# Iterate through the DataFrame
for i in range(df.shape[0]):
    current_set = df.loc[i, 'set']
    count = 0  # Initialize count for current row
    # Iterate backwards to check previous rows for same set until a miss is found
    for j in range(i - 1, -1, -1):  # Loop from the previous row to the beginning
        if df.loc[j, 'hit/miss'] == 0:  # Break on encountering a miss
                break
        else:
            if df.loc[j, 'set'] == current_set:  # Match current set
                count += 1  # Increment count for matching set access
    salm.append(count)

# Add the results as a new column in the DataFrame
df['salm'] = salm

print(df)


      time_stamp      pc       address         tag offset  set way  \
0       90161376  454b26  464939bbe750  0x23249cdd   0x10  79d   3   
1       90161377  454b26  464939bbf670  0x23249cdd   0x30  7d9   7   
2       90161378  454b26  464939bbfbc0  0x23249cdd    0x0  7ef   4   
3       90161432  454b26  464939bbb790  0x23249cdd   0x10  6de   3   
4       90161483  454b26  464939bbdfb0  0x23249cdd   0x30  77e   f   
...          ...     ...           ...         ...    ...  ...  ..   
9995    90867017  454b26  1a04b5770530   0xd025abb   0x30  414   b   
9996    90867032  454b26  1a04b5771930   0xd025abb   0x30  464   a   
9997    90867033  454b26  1a04b57720f0   0xd025abb   0x30  483   b   
9998    90867189  454b1e  afd74f9ffe10  0x57eba7cf   0x10  7f8   e   
9999    90867289  454b1e  afd74f9ffe50  0x57eba7cf   0x10  7f9   b   

      instr_type  recency  hit/miss 3_tag_1 3_tag_2 3_tag_3  hits_count  salm  
0            0.0      9.0       1.0     0x5     0x3     0x3           0     0  

In [209]:
# CODE FOR SET ACCESSES SINCE LAST MISS

############# EXAMPLE ###################

# Example DataFrame
data = {
    'set': [1, 2, 3, 1, 2, 1, 2, 1, 3, 3, 1, 1, 2, 1],
    'hit/miss': [0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1]
}
df_trial = pd.DataFrame(data)

# Initialize a results list
salm = []

# Iterate through the DataFrame
for i in range(df_trial.shape[0]):
    current_set = df_trial.loc[i, 'set']
    count = 0  # Initialize count for current row
    flag=False
    # Iterate backwards to check previous rows for same set until a miss is found
    for j in range(i - 1, -1, -1):  # Loop from the previous row to the beginning
        if df_trial.loc[j, 'hit/miss'] == 0:  # Break on encountering a miss
                break
        else:
            if df_trial.loc[j, 'set'] == current_set:  # Match current set
                count += 1  # Increment count for matching set access
    salm.append(count)

# Add the results as a new column in the DataFrame
df_trial['salm'] = salm

print(df_trial)


    set  hit/miss  salm
0     1         0     0
1     2         1     0
2     3         0     0
3     1         1     0
4     2         1     0
5     1         1     1
6     2         1     1
7     1         1     2
8     3         0     0
9     3         0     0
10    1         1     0
11    1         1     1
12    2         1     0
13    1         1     2


In [ ]:
# CODE FOR PREUSE DISTANCES

############### EXAMPLE ################

# Sample DataFrame
data = {
    'address': [1, 2, 3, 1, 2, 1, 2, 1],
    'set': [0, 1, 0, 1, 1, 1, 1, 1]
}
df_trial = pd.DataFrame(data)

# Initialize a results list
results = []

# Iterate through the DataFrame
for i in range(df_trial.shape[0]):
    current_address = df_trial.loc[i, 'address']
    current_set = df_trial.loc[i, 'set']
    preuse_distance = 0  # Default preuse distance is 0
    
    # Iterate backwards to find the last matching address
    for j in range(i - 1, -1, -1):  # Loop from the previous row to the beginning
        if df_trial.loc[j, 'address'] == current_address:
            # Count the occurrences of the current set between j and i
            preuse_distance = sum(df_trial.loc[k, 'set'] == current_set for k in range(j + 1, i))
            break  # Stop once the last matching address is found

    # Append the preuse distance to the results
    results.append(preuse_distance)

# Add the results as a new column in the DataFrame
df_trial['preuse_distance'] = results

print(df_trial)